In [ ]:
import sys
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import scipy.integrate
import math
import pandas as pd
import itertools
import random
import statsmodels.api as sm
import time

############
from Baseline_Functions_Definitions import z_expectation_variance
from Experiment_Framework import product_diff_list,question_extractor,norm_AO_MO_data_generation
from Questionnaire_Procedure import moment_matching_update
from Batch_Design_and_Rollout import batch_design_AO, batch_design_MO

In [ ]:
#This function is used to generate an nxn matrix M(r) such that M_ij = r^|i-j| for -1<r<1. This matrix is called a 
#Kac-Murdock-Szego matrix.
def KMS_Matrix(n,r):
    #n: this is the number of rows and columns of the matrix
    #r: this is the coefficient given above that determines the value of the matrice's entries 
    
    
    M = np.zeros([n,n])
    for i in range(n):
        for j in range(n):
            M[i,j] = r**abs(i-j)
        
    return M

In [ ]:
#This function is used to create a set of batch designs and evaluate their D-error (in a sequential manner). 
#We save the D-error of each batch in a list.

def random_batch_D_error(init_mu,init_Sig,batch_size,num_random_batches,true_partworths, gumbel_error_terms):
    #init_mu: This is the initial expectation of the partworths.
    #init_Sig: This is the initial covariance matrix of the partworths.
    #batch_size: This is the number of questions in each batch.
    #num_random_batches: This is the number of random batches that we will generate.
    #true_partworths: This is the true/baseline partworths we will use to evaluate the d-error of a design.
    #gumbel_error_terms: This is a list of gumbel errors terms used in evaluating the d-error of a design. This 
    #list should have dimension (true_partworths x batch_size), where each entry is a list with two randomly generated
    #gumbel terms. We use the same true partworths and error terms for evaluating the d-error of each randomly generated 
    #design.
    
    attr_num = len(init_mu)
    
    average_d_error = []
    
    #Create a list of all products.
    prod_list = product_diff_list(attr_num)
    
    #Construct the set of batch designs
    batch_set = [[] for i in range(num_random_batches)]
    for i in range(num_random_batches):
        random_question_matrix = random.sample(prod_list,batch_size)
        for m in range(batch_size):
            #random_question_vec = random.sample(prod_list,1)[0]
            #[x,y] = question_extractor(random_question_vec)
            [x,y] = question_extractor(random_question_matrix[m])
            batch_set[i].append([x,y])
        
            
    #Evaluate the d-error
    #true_partworths = []
    #for t in range(num_true_partworths):
                #true_partworths.append(rng.multivariate_normal(init_mu,init_Sig))
    num_true_partworths = len(true_partworths)
    
    for i in range(num_random_batches):
                #Create a list for the batch that will store the final determinant value for each simulation
                #corresponding to each baseline partworth.
                batch_simulate_d_values = []
                #print('random_batch_number: ' + str(i))
                #Simulate d-efficiency over baseline partworths
                for j in range(num_true_partworths):
                #Each time we start with a new partworth, we must use the initial prior parameters.
                    mu = init_mu
                    Sig = init_Sig
                    
                    #Each simulation goes through the questions in the random batch.
                    for k in range(batch_size):
                    #Set x and y
                        x = batch_set[i][k][0]
                        y = batch_set[i][k][1]
                
                        #These temp variables will be used in the choice model below in case the user prefers y over x.
                        x_temp = x
                        y_temp = y
                        
                        #See preference between two products.
                        gum_x = gumbel_error_terms[j][k][0]#np.random.gumbel(0,1)
                        gum_y = gumbel_error_terms[j][k][1]#np.random.gumbel(0,1)
                        if (np.dot(true_partworths[j],np.array(y)) + gum_y) >= (np.dot(true_partworths[j],np.array(x)) + gum_x):
                            x = y_temp
                            y = x_temp
                            
                        #Perform moment matching after choice is made.
                        [mu, Sig] = moment_matching_update(x,y,mu,Sig)
                        
                    #After the questionnaire for a baseline partworth is complete, we append the square root of the determinant
                    #of the final covariance matrix.
                    batch_simulate_d_values.append(np.sqrt(np.linalg.det(Sig)))
                    
                #We average the d-values from the simulation for a batch and store it in a list.
                average_d_error.append(np.mean(batch_simulate_d_values))
                
    return average_d_error

In [ ]:
#Fitting the models batch_AO and batch_MO.
rng = np.random.default_rng(100) 
np.random.seed(100)
random.seed(100)

#signal to noise ratio. 
#1 - LOW: multiply expectation by 0.25 and covariance by 4.0
#2 - REG: multiply expectation by 1.0 and covariance by 1.0
#3 - HIGH: multiply expectation by 4.0 and covariance by 0.25
snr = int(sys.argv[1])


#Prior type
#1 - homogeneous expectation and identity covariance matrix
#2 - heterogeneous expectation and KMS covariance matrix
prior_type = int(sys.argv[2])

if snr == 1:
    if prior_type == 1:
        init_mu_fit = 0.25*np.array(6*[1.0])
        init_Sig_fit = 4.0*np.identity(6)
    if prior_type == 2:
        init_mu_fit = 0.25*np.array([-0.25,0.5,-0.75,1.0,-1.25,1.5])
        init_Sig_fit = 4.0*(1.25*KMS_Matrix(6,-0.5))
if snr == 2:
    if prior_type == 1:
        init_mu_fit = 1.0*np.array(6*[1.0])
        init_Sig_fit = 1.0*np.identity(6)
    if prior_type == 2:
        init_mu_fit = 1.0*np.array([-0.25,0.5,-0.75,1.0,-1.25,1.5])
        init_Sig_fit = 1.0*(1.25*KMS_Matrix(6,-0.5))
if snr == 3:
    if prior_type == 1:
        init_mu_fit = 4.0*np.array(6*[1.0])
        init_Sig_fit = 0.25*np.identity(6)
    if prior_type == 2:
        init_mu_fit = 4.0*np.array([-0.25,0.5,-0.75,1.0,-1.25,1.5])
        init_Sig_fit = 0.25*(1.25*KMS_Matrix(6,-0.5))
    
batch_size_fit = 4
    
L_fit = [1.0]
S_fit = [1.0]

num_random_batches_fit = 1000
num_true_partworths_fit = 100

In [ ]:
#Generate the data in order to estimate the parameters of the AO and MO models
average_orthogonality_fit, maximum_orthogonality_fit, average_question_mean_fit, average_question_variance_fit, L_mu_fit, S_Sig_fit, init_sqrt_determinant_fit, average_d_error_fit = norm_AO_MO_data_generation(init_mu_fit, init_Sig_fit, batch_size_fit, L_fit, S_fit, num_random_batches_fit, num_true_partworths_fit)

In [ ]:
#Create a dataframe of the generated data for fitting the parameters of the AO and MO models
df_fit = pd.DataFrame(list(zip(average_orthogonality_fit, maximum_orthogonality_fit, average_question_mean_fit, average_question_variance_fit, L_mu_fit, S_Sig_fit, init_sqrt_determinant_fit, average_d_error_fit)),
                  columns =['Avg_Orth', 'Max_Orth', 'Avg_Quest_Mean', 'Avg_Quest_Var', 'L_mu_norm', 'S_Sig_norm', 'Init_Sqrt_Det', 'D_err'])

In [ ]:
#Add some new columns to the dataset. We mean-center the independent variables to attempt to reduce VIF. This will not affect the value of
#of the coefficients, except for the intercept.
df_fit['log_norm_derr'] = np.log(np.divide(np.array(df_fit['D_err']),np.array(df_fit['Init_Sqrt_Det'])))
df_fit['cent_norm_AM'] = np.divide(np.array(df_fit['Avg_Quest_Mean']),np.array(df_fit['L_mu_norm'])) - np.mean(np.divide(np.array(df_fit['Avg_Quest_Mean']),np.array(df_fit['L_mu_norm'])))
df_fit['cent_norm_AV'] = np.divide(np.array(df_fit['Avg_Quest_Var']),np.array(df_fit['S_Sig_norm'])) - np.mean(np.divide(np.array(df_fit['Avg_Quest_Var']),np.array(df_fit['S_Sig_norm'])))
df_fit['cent_norm_AO'] = np.divide(np.array(df_fit['Avg_Orth']),np.array(df_fit['S_Sig_norm'])) - np.mean(np.divide(np.array(df_fit['Avg_Orth']),np.array(df_fit['S_Sig_norm'])))
df_fit['cent_norm_MO'] = np.divide(np.array(df_fit['Max_Orth']),np.array(df_fit['S_Sig_norm'])) - np.mean(np.divide(np.array(df_fit['Max_Orth']),np.array(df_fit['S_Sig_norm'])))

df_fit['cent_L_mu_norm'] = df_fit['L_mu_norm'] - np.mean(np.array(df_fit['L_mu_norm']))
df_fit['cent_S_Sig_norm'] = df_fit['S_Sig_norm'] - np.mean(np.array(df_fit['S_Sig_norm']))

In [ ]:
#Save resulting file as a CSV.
if snr == 1:
    if prior_type == 1:
        df_fit.to_csv('MIPvEnum_Normalized_AO_MO_Model_Data_mu025_Sig4Ident_batchsize4_L_1_S_1_nrb_1000_ntp_100_v6.csv')
    if prior_type == 2:
        df_fit.to_csv('MIPvEnum_Normalized_AO_MO_Model_Data_lowsnr_muhet_SigKMS_batchsize4_L_1_S_1_nrb_1000_ntp_100_v6.csv')
if snr == 2:
    if prior_type == 1:
        df_fit.to_csv('MIPvEnum_Normalized_AO_MO_Model_Data_mu1_Sig1Ident_batchsize4_L_1_S__1_nrb_1000_ntp_100_v6.csv')
    if prior_type == 2:
        df_fit.to_csv('MIPvEnum_Normalized_AO_MO_Model_Data_medsnr_muhet_SigKMS_batchsize4_L_1_S_1_nrb_1000_ntp_100_v6.csv')
if snr == 3:
    if prior_type == 1:
        df_fit.to_csv('MIPvEnum_Normalized_AO_MO_Model_Data_mu4_Sig025Ident_batchsize4_L_1_S_1_nrb_1000_ntp_100_v6.csv')
    if prior_type == 2:
        df_fit.to_csv('MIPvEnum_Normalized_AO_MO_Model_Data_highsnr_muhet_SigKMS_batchsize4_L_1_S_1_nrb_1000_ntp_100_v6.csv')

In [ ]:
#Model with AO
#Since we use L=S=[1.0], we do not need the regression terms L_mu_norm and S_Sig_norm
model_AO = sm.formula.ols(formula = "log_norm_derr ~  cent_norm_AM + cent_norm_AV + cent_norm_AO", data = df_fit).fit()
parameter_est_AO = model_AO.params

In [ ]:
#Model with MO
#Since we use L=S=[1.0], we do not need the regression terms L_mu_norm and S_Sig_norm
model_MO = sm.formula.ols(formula = "log_norm_derr ~  cent_norm_AM + cent_norm_AV + cent_norm_MO", data = df_fit).fit()
parameter_est_MO = model_MO.params

In [ ]:
#Save model ouput as a text file
if snr == 1:
    if prior_type == 1:
        with open('summary_low_homo_AO_v6.txt', 'w') as fh:
            fh.write(model_AO.summary().as_text())
        with open('summary_low_homo_MO_v6.txt', 'w') as fh:
            fh.write(model_MO.summary().as_text())
    if prior_type == 2:
        with open('summary_low_het_AO_v6.txt', 'w') as fh:
            fh.write(model_AO.summary().as_text())
        with open('summary_low_het_MO_v6.txt', 'w') as fh:
            fh.write(model_MO.summary().as_text())
if snr == 2:
    if prior_type == 1:
        with open('summary_med_homo_AO_v6.txt', 'w') as fh:
            fh.write(model_AO.summary().as_text())
        with open('summary_med_homo_MO_v6.txt', 'w') as fh:
            fh.write(model_MO.summary().as_text())
    if prior_type == 2:
        with open('summary_med_het_AO_v6.txt', 'w') as fh:
            fh.write(model_AO.summary().as_text())
        with open('summary_med_het_MO_v6.txt', 'w') as fh:
            fh.write(model_MO.summary().as_text())
if snr == 3:
    if prior_type == 1:
        with open('summary_high_homo_AO_v6.txt', 'w') as fh:
            fh.write(model_AO.summary().as_text())
        with open('summary_high_homo_MO_v6.txt', 'w') as fh:
            fh.write(model_MO.summary().as_text())
    if prior_type == 2:
        with open('summary_high_het_AO_v6.txt', 'w') as fh:
            fh.write(model_AO.summary().as_text())
        with open('summary_high_het_MO_v6.txt', 'w') as fh:
            fh.write(model_MO.summary().as_text())

In [ ]:
#Save model parameters in txt file
if snr == 1:
    params_est = np.array([parameter_est_AO[1],parameter_est_AO[2],parameter_est_AO[3],parameter_est_MO[1],parameter_est_MO[2],parameter_est_MO[3]])
    if prior_type == 1:
        np.savetxt('MIPvEnum_modelparams_lowsnr_v6.txt',params_est)
    if prior_type == 2:
        np.savetxt('MIPvEnum_modelparams_lowsnr_muhet_SigKMS_v6.txt', params_est)
if snr == 2:
    params_est = np.array([parameter_est_AO[1],parameter_est_AO[2],parameter_est_AO[3],parameter_est_MO[1],parameter_est_MO[2],parameter_est_MO[3]])
    if prior_type == 1:
        np.savetxt('MIPvEnum_modelparams_medsnr_v6.txt',params_est)
    if prior_type == 2:
        np.savetxt('MIPvEnum_modelparams_medsnr_muhet_SigKMS_v6.txt', params_est)
if snr == 3:
    params_est = np.array([parameter_est_AO[1],parameter_est_AO[2],parameter_est_AO[3],parameter_est_MO[1],parameter_est_MO[2],parameter_est_MO[3]])
    if prior_type == 1:
        np.savetxt('MIPvEnum_modelparams_highsnr_v6.txt',params_est)
    if prior_type == 2:
        np.savetxt('MIPvEnum_modelparams_highsnr_muhet_SigKMS_v6.txt', params_est)

In [ ]:
#Experiment settings:
#Settings for experiment
rng = np.random.default_rng(100)
np.random.seed(100)
random.seed(100)

    
if snr == 1:
    if prior_type == 1:
        mu_exp = 0.25*np.array(6*[1.0])
        Sig_exp = 4.0*np.identity(6)
    if prior_type == 2:
        mu_exp = 0.25*np.array([-0.25,0.5,-0.75,1.0,-1.25,1.5])
        Sig_exp = 4.0*(1.25*KMS_Matrix(6,-0.5))
if snr == 2:
    if prior_type == 1:
        mu_exp = 1.0*np.array(6*[1.0])
        Sig_exp = 1.0*np.identity(6)
    if prior_type == 2:
        mu_exp = 1.0*np.array([-0.25,0.5,-0.75,1.0,-1.25,1.5])
        Sig_exp = 1.0*(1.25*KMS_Matrix(6,-0.5))
if snr == 3:
    if prior_type == 1:
        mu_exp = 4.0*np.array(6*[1.0])
        Sig_exp = 0.25*np.identity(6)
    if prior_type == 2:
        mu_exp = 4.0*np.array([-0.25,0.5,-0.75,1.0,-1.25,1.5])
        Sig_exp = 0.25*(1.25*KMS_Matrix(6,-0.5))

batch_size_exp = 4
num_random_batches_exp = 100000       

true_partworth_exp = []

num_true_partworths_exp = 100
#There will be 100 true partworths
for t in range(num_true_partworths_exp):
    true_partworth_exp.append(rng.multivariate_normal(mu_exp,Sig_exp))
    
#Create list of gumbel error terms
gumbel_error_terms_exp = [[[np.random.gumbel(0,1) for k in range(2)] for j in range(batch_size_exp)] for i in range(num_true_partworths_exp)]
print(gumbel_error_terms_exp[0])   

AO_alpha_exp = parameter_est_AO[1]
AO_kappa_exp = parameter_est_AO[2]
AO_gamma_exp = parameter_est_AO[3]

MO_alpha_exp = parameter_est_MO[1]
MO_kappa_exp = parameter_est_MO[2]
MO_gamma_exp = parameter_est_MO[3]



In [ ]:
#Create list of d-error of random batch designs
tic = time.perf_counter()
d_err_list = random_batch_D_error(mu_exp,Sig_exp,batch_size_exp,num_random_batches_exp,true_partworth_exp,gumbel_error_terms_exp)
toc = time.perf_counter()
time_meas = toc - tic

In [ ]:
#Create Batch_AO design and evaluate its D-error
[AO_batch,AO_Ortho] = batch_design_AO(mu_exp,Sig_exp,batch_size_exp,AO_alpha_exp,AO_kappa_exp,AO_gamma_exp,t_lim = 100,logfile = True)

In [ ]:
#Create Batch_MO design and evaluate its D-error
[MO_batch, MO_Ortho] = batch_design_MO(mu_exp,Sig_exp,batch_size_exp,MO_alpha_exp,MO_kappa_exp,MO_gamma_exp,t_lim = 100,logfile = True)

In [ ]:
#Evaluate the d-error of the Batch_AO design

#Create a list for the batch that will store the final determinant value for each simulation
#corresponding to each baseline partworth.\

batch_AO_d_values = []
                
#Simulate d-efficiency over baseline partworths
for j in range(len(true_partworth_exp)):
    #Each time we start with a new partworth, we must use the initial prior parameters.
    mu_AO = mu_exp
    Sig_AO = Sig_exp
                    
    #Each simulation goes through the questions in the random batch.
    for k in range(batch_size_exp):
        #Set x and y
        x_AO = AO_batch[k][0]
        y_AO = AO_batch[k][1]
                
        #These temp variables will be used in the choice model below in case the user prefers y over x.
        x_temp_AO = x_AO
        y_temp_AO = y_AO
        
        gum_x_AO = gumbel_error_terms_exp[j][k][0]
        gum_y_AO = gumbel_error_terms_exp[j][k][1]
        #See preference between two products.
        if (np.dot(true_partworth_exp[j],np.array(y_AO)) + gum_y_AO) >= (np.dot(true_partworth_exp[j],np.array(x_AO)) + gum_x_AO):
            x_AO = y_temp_AO
            y_AO = x_temp_AO
                            
        #Perform moment matching after choice is made.
        [mu_AO, Sig_AO] = moment_matching_update(x_AO,y_AO,mu_AO,Sig_AO)
                        
    #After the questionnaire for a baseline partworth is complete, we append the square root of the determinant
    #of the final covariance matrix.
    batch_AO_d_values.append(np.sqrt(np.linalg.det(Sig_AO)))
                    
#We average the d-values from the simulation for the AO batch and save it as a variable.
batch_AO_d_error = np.mean(batch_AO_d_values)

In [ ]:
#Evaluate the d-error of the Batch_MO design

#Create a list for the batch that will store the final determinant value for each simulation
#corresponding to each baseline partworth.
batch_MO_d_values = []
                
#Simulate d-efficiency over baseline partworths
for j in range(len(true_partworth_exp)):
    #Each time we start with a new partworth, we must use the initial prior parameters.
    mu_MO = mu_exp
    Sig_MO = Sig_exp
                    
    #Each simulation goes through the questions in the random batch.
    for k in range(batch_size_exp):
        #Set x and y
        x_MO = MO_batch[k][0]
        y_MO = MO_batch[k][1]
                
        #These temp variables will be used in the choice model below in case the user prefers y over x.
        x_temp_MO = x_MO
        y_temp_MO = y_MO
        
        gum_x_MO = gumbel_error_terms_exp[j][k][0]
        gum_y_MO = gumbel_error_terms_exp[j][k][1]
        #See preference between two products. 
        if (np.dot(true_partworth_exp[j],np.array(y_MO)) + gum_y_MO) >= (np.dot(true_partworth_exp[j],np.array(x_MO)) + gum_x_MO):
            x_MO = y_temp_MO
            y_MO = x_temp_MO
                            
        #Perform moment matching after choice is made.
        [mu_MO, Sig_MO] = moment_matching_update(x_MO,y_MO,mu_MO,Sig_MO)
                        
    #After the questionnaire for a baseline partworth is complete, we append the square root of the determinant
    #of the final covariance matrix.
    batch_MO_d_values.append(np.sqrt(np.linalg.det(Sig_MO)))
                    
#We average the d-values from the simulation for the AO batch and save it as a variable.
batch_MO_d_error = np.mean(batch_MO_d_values)

In [ ]:
batch_AO_success = [1 if batch_AO_d_error < rand_batch_d_error else 0 for rand_batch_d_error in d_err_list]
batch_MO_success = [1 if batch_MO_d_error < rand_batch_d_error else 0 for rand_batch_d_error in d_err_list]

In [ ]:
#Save d-error information and success information in a csv
batch_information_list = np.array([d_err_list,batch_AO_success,batch_MO_success]).T
batch_information_list_df = pd.DataFrame(batch_information_list, columns = ['D-errors', 'Batch AO < D-error', 'Batch MO < D-error'])

if snr == 1:
    if prior_type == 1:
        batch_information_list_df.to_csv('MIPvsEnum_lowsnr_d_error_batch_comparison_list_v6.csv')
    if prior_type == 2:
        batch_information_list_df.to_csv('MIPvsEnum_lowsnr_d_error_batch_comparison_list_muhet_SigKMS_v6.csv')
if snr == 2:
    if prior_type == 1:
        batch_information_list_df.to_csv('MIPvsEnum_mediumsnr_d_error_batch_comparison_list_v6.csv')
    if prior_type == 2:
        batch_information_list_df.to_csv('MIPvsEnum_mediumsnr_d_error_batch_comparison_list_muhet_SigKMS_v6.csv')
if snr == 3:
    if prior_type == 1:
        batch_information_list_df.to_csv('MIPvsEnum_highsnr_d_error_batch_comparison_list_v6.csv')
    if prior_type == 2:
        batch_information_list_df.to_csv('MIPvsEnum_highsnr_d_error_batch_comparison_list_muhet_SigKMS_v6.csv')

In [ ]:
#Construct estimates and confidence interval
AO_prop_est = np.mean(batch_AO_success)
MO_prop_est = np.mean(batch_MO_success)

AO_CI_upper = AO_prop_est + 1.96*np.sqrt(AO_prop_est*(1-AO_prop_est)/len(batch_AO_success))
AO_CI_lower = AO_prop_est - 1.96*np.sqrt(AO_prop_est*(1-AO_prop_est)/len(batch_AO_success))

MO_CI_upper = MO_prop_est + 1.96*np.sqrt(MO_prop_est*(1-MO_prop_est)/len(batch_MO_success))
MO_CI_lower = MO_prop_est - 1.96*np.sqrt(MO_prop_est*(1-MO_prop_est)/len(batch_MO_success))

In [ ]:
#Save confidence interval information, time for iterating over random batches, and d-error of batches
if snr == 1:
    CI_information = np.array(['batch_AO_d_error: ' + str(batch_AO_d_error),'AO_prop_est: ' +str(AO_prop_est),'AO_CI_lower: ' + str(AO_CI_lower),'AO_CI_upper: ' + str(AO_CI_upper),'batch_MO_d_error: ' + str(batch_MO_d_error),'MO_prop_est: ' + str(MO_prop_est),'MO_CI_lower:' + str(MO_CI_lower),'MO_CI_upper: ' + str(MO_CI_upper),'time for enumeration: ' + str(time_meas)])
    if prior_type == 1:
        np.savetxt('MIPvEnum_CIinfo_lowsnr_v6.txt',CI_information,fmt='%s')
    if prior_type == 2:
        np.savetxt('MIPvEnum_CIinfo_lowsnr_muhet_SigKMS_v6.txt',CI_information,fmt='%s')
if snr == 2:
    CI_information = np.array(['batch_AO_d_error: ' + str(batch_AO_d_error),'AO_prop_est: ' +str(AO_prop_est),'AO_CI_lower: ' + str(AO_CI_lower),'AO_CI_upper: ' + str(AO_CI_upper),'batch_MO_d_error: ' + str(batch_MO_d_error),'MO_prop_est: ' + str(MO_prop_est),'MO_CI_lower:' + str(MO_CI_lower),'MO_CI_upper: ' + str(MO_CI_upper),'time for enumeration: ' + str(time_meas)])
    if prior_type == 1:
        np.savetxt('MIPvEnum_CIinfo_mediumsnr_v6.txt',CI_information,fmt='%s')
    if prior_type == 2:
        np.savetxt('MIPvEnum_CIinfo_mediumsnr_muhet_SigKMS_v6.txt',CI_information,fmt='%s')
if snr == 3:
    CI_information = np.array(['batch_AO_d_error: ' + str(batch_AO_d_error),'AO_prop_est: ' +str(AO_prop_est),'AO_CI_lower: ' + str(AO_CI_lower),'AO_CI_upper: ' + str(AO_CI_upper),'batch_MO_d_error: ' + str(batch_MO_d_error),'MO_prop_est: ' + str(MO_prop_est),'MO_CI_lower:' + str(MO_CI_lower),'MO_CI_upper: ' + str(MO_CI_upper),'time for enumeration: ' + str(time_meas)])
    if prior_type == 1:
        np.savetxt('MIPvEnum_CIinfo_highsnr_v6.txt',CI_information,fmt='%s')
    if prior_type == 2:
        np.savetxt('MIPvEnum_CIinfo_highsnr_muhet_SigKMS_v6.txt',CI_information,fmt='%s')

In [ ]:
#Save information regarding the batch design.
if snr == 1:
    batch_info = np.array(['AO_batch: '+str(AO_batch),'AO_Orth: '+str(AO_Ortho),'MO_batch: '+str(MO_batch),'MO_Orth: '+str(MO_Ortho)])
    if prior_type == 1:
        np.savetxt('MIPvEnum_batchinfo_lowsnr_v6.txt',batch_info,fmt='%s')
    if prior_type == 2:
        np.savetxt('MIPvEnum_batchinfo_lowsnr_muhet_SigKMS_v6.txt',batch_info,fmt='%s')
if snr == 2:
    batch_info = np.array(['AO_batch: '+str(AO_batch),'AO_Orth: '+str(AO_Ortho),'MO_batch: '+str(MO_batch),'MO_Orth: '+str(MO_Ortho)])
    if prior_type == 1:
        np.savetxt('MIPvEnum_batchinfo_mediumsnr_v6.txt',batch_info,fmt='%s')
    if prior_type == 2:
        np.savetxt('MIPvEnum_batchinfo_mediumsnr_muhet_SigKMS_v6.txt',batch_info,fmt='%s')
if snr == 3:
    batch_info = np.array(['AO_batch: '+str(AO_batch),'AO_Orth: '+str(AO_Ortho),'MO_batch: '+str(MO_batch),'MO_Orth: '+str(MO_Ortho)])
    if prior_type == 1:
        np.savetxt('MIPvEnum_batchinfo_highsnr_v6.txt',batch_info,fmt='%s')
    if prior_type == 2:
        np.savetxt('MIPvEnum_batchinfo_highsnr_muhet_SigKMS_v6.txt',batch_info,fmt='%s')